In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22,
                     'ytick.labelsize': 20,
                     'xtick.labelsize': 20})

In [2]:
# Separate the dataset into features and labels.
# Included header = None so that the first row is not considered as feature names.
dataset = pd.read_csv('mnist.csv', header=None)

# Get X
X = dataset.iloc[:,1:]

# Get y
y = dataset.iloc[:,0]

### Performing PCA to get 10 dimensional feature vector

In [3]:
# Import the pca model
from sklearn.decomposition import PCA

# Get matrix with 10 principle components.
pca = PCA(n_components=10)
X_transformed = pca.fit_transform(X)

### Get Covariance matrix of transformed data

In [4]:
'''
PCA done on the transformed data again so that the covariance matrix can be obtained from the pca object.
'''
pca_transformed = PCA()
pca_transformed.fit_transform(X_transformed)
covariance_matrix_transformed = pca_transformed.get_covariance()
cov_mat_df = pd.DataFrame(covariance_matrix_transformed)

print("The covariance matrix is:")
cov_mat_df

The covariance matrix is:


,0,1,2,3,4,5,6,7,8,9
0,5.305587,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.000000,3.877016e+00,-4.533762e-16,-1.421930e-15,-3.552495e-16,4.952152e-16,1.398757e-15,-8.010797e-17,-5.615156e-16,-4.729745e-16
2,0.000000,-4.533762e-16,3.287047e+00,6.216716e-16,1.603012e-15,-8.518867e-16,-1.630443e-15,3.549769e-16,4.825069e-16,-6.942499e-16
3,0.000000,-1.421930e-15,6.216716e-16,2.912541e+00,2.519613e-14,-1.580395e-16,1.152433e-15,1.065975e-15,6.382674e-16,-7.936584e-16
4,0.000000,-3.552495e-16,1.603012e-15,2.519613e-14,2.486332e+00,-5.076640e-16,-2.168132e-16,3.253270e-16,-8.738259e-16,1.193177e-16
5,0.000000,4.952152e-16,-8.518867e-16,-1.580395e-16,-5.076640e-16,2.353589e+00,-1.060327e-15,-3.989108e-16,3.724444e-16,9.351751e-16
6,0.000000,1.398757e-15,-1.630443e-15,1.152433e-15,-2.168132e-16,-1.060327e-15,1.755568e+00,1.590555e-15,1.265226e-15,-2.864769e-16
7,0.000000,-8.010797e-17,3.549769e-16,1.065975e-15,3.253270e-16,-3.989108e-16,1.590555e-15,1.544760e+00,3.085506e-16,-5.044808e-16
8,0.000000,-5.615156e-16,4.825069e-16,6.382674e-16,-8.738259e-16,3.724444e-16,1.265226e-15,3.085506e-16,1.454744e+00,-5.728881e-16
9,0.000000,-4.729745e-16,-6.942499e-16,-7.936584e-16,1.193177e-16,9.351751e-16,-2.864769e-16,-5.044808e-16,-5.728881e-16,1.243368e+00


### Sum of covariance matrix

In [5]:
sum_cov = np.sum(covariance_matrix_transformed)
print("The sum of the covariance matrix is: ", sum_cov)

The sum of the covariance matrix is:  26.22055181603834


### K means clustering

In [6]:
from sklearn.cluster import KMeans

'''
Initializing and model fitting. While fitting, only the features were passed since K-means is an unsupervised
classifier.
'''
k_means_model = KMeans(n_clusters=10, random_state=1)
fitted_model = k_means_model.fit(X)

### Showing centroids

In [7]:
'''
Since there are 784 features, each centroid point coordinate will have 784 entries. Therefore, the centroid_points
array will be of shape 10 x 784.
'''
centroid_points = fitted_model.cluster_centers_
print("The centroid points are:\n", centroid_points)

The centroid points are:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Summation of all dimensions for each centroid point

In [8]:
summations = [np.sum(each) for each in centroid_points]
print("The centroid sums are:\n", summations)

The centroid sums are:
 [69.53379772961816, 112.09345113539487, 102.12228349673202, 119.69496911914612, 95.91205020469727, 65.78649237472769, 126.74630178210371, 94.53241766916106, 146.29338286893704, 118.50340417628485]


### Loss curve

In [ ]:
max_iterations = fitted_model.max_iter
losses = []
iterations = np.arange(1, max_iterations + 1)
for iteration_count in iterations:
    model = KMeans(n_clusters=10, random_state=1, max_iter=iteration_count)
    fitted_model_temp = model.fit(X)
    
    # Append inertia / loss for the model for iteration.
    losses.append(fitted_model_temp.inertia_)
    
# Plot loss vs cluster count
plt.figure(figsize=(30, 15))
plt.plot(iterations, losses)
plt.title('Loss for K-means')
plt.xlabel('Number iterations')
plt.ylabel('Loss')
plt.show()

### Optimal K identification using elbow method

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, silhouette_score
import math

# Test train splits.
X_train = X[0:4000]
X_val = X[4000:]
y_train = y[0:4000]
y_val = y[4000:]

'''
The following code attempts to determine the best value of k via the elbow method. But, the loss vs clusters curve
is so smooth that it is hard to determine the elbow point (point from which the maximum reduction in gradient takes
place) As a result a range was pointed out in the graph within which the optimal value of K was suspected to be in.
'''

# Plotting graph of number of clusters vs loss for elbow method.
clusters = np.arange(2, 31)
losses = []
for cluster_count in clusters:
    model = KMeans(n_clusters=cluster_count, random_state=1)
    fitted_model_temp = model.fit(X_train)
    
    # Append inertia / loss for the model for iteration.
    losses.append(fitted_model_temp.inertia_)
    
# Plot loss vs cluster count
plt.figure(figsize=(30, 15))
plt.plot(clusters, losses, color='blue')
plt.axvline(x = 6, color = 'black')
plt.axvline(x = 10, color = 'black')
plt.legend(['Training', 'optimal K range'])
plt.title('loss vs number of clusters')
plt.xticks(ticks=clusters)
plt.xlabel('Number of clusters')
plt.ylabel('Loss')
plt.show()

### Optimal K identification using silhouette score

In [10]:
'''
The following code attempts to determine the best value of k via the use of Silhouette coefficient. This is because
the elbow method does not give us a clear indication of what the K value exactly is. In the silhouette coefficient
graph, the optimal value of K has been identified with a vertical line.
'''

silhouette_scores_train = []
silhouette_scores_val = []

for cluster_count in clusters:
    model = KMeans(n_clusters=cluster_count, random_state=1)
    fitted_model_temp = model.fit(X_train)
    labels = fitted_model_temp.labels_
    print(labels)
    break
    val_predictions = fitted_model_temp.predict(X_val)
    score_train = silhouette_score(X_train, labels)
    score_val = silhouette_score(X_val, val_predictions)
    
    # Append validation accuracy scores along with the corresponding cluster count.
    silhouette_scores_train.append(score_train)
    silhouette_scores_val.append(score_val)

    
# Plot Accuracy vs cluster count
plt.figure(figsize=(30, 15))
plt.plot(clusters, silhouette_scores_train, color = 'blue')
plt.plot(clusters, silhouette_scores_val, color = 'red')
plt.axvline(x = 8, color = 'black')
plt.legend(['Training', 'Validation', 'optimal K'])
plt.xticks(ticks=clusters)
plt.title('Accuracy vs number of clusters')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.show()

[1 1 0 ... 1 1 1]


### K-means with rbf kernel

In [11]:
# First 500 samples of the original dataset.
X_5 = X[0:500]

# A custom k-means class which applies RBF kernel on the data.
class KMeans_custom:
    def fit(self, x):
        # If less samples than features, select number of samples as the dimension of K.
        shape = x.shape
        N = min(shape)
        k = np.zeros((N, N))
        
        # Compute 2 sigma squared using the formula provided.
        two_sigma_squared = 0
        for i in range(N):
            for j in range(N):
                distance = pow(np.linalg.norm((x[i] - x[j])), 2)
                two_sigma_squared += distance
        two_sigma_squared = two_sigma_squared / pow(N, 2)
        
        # Compute the value of the kernel output.
        for i in range(N):
            for j in range(N):
                numerator = -pow(np.linalg.norm((x[i] - x[j])), 2)
                k[i][j] = np.exp(numerator / two_sigma_squared)
        self.kernel_output = k
        
        # Use Sklearn kmeans, but provide kernel output as input.
        self.model = KMeans(n_clusters=5, random_state=1).fit(self.kernel_output)
        
    def get_centroids(self):
        return self.model.cluster_centers_
    
    
kmeans_model_rbf = KMeans_custom()
kmeans_model_rbf.fit(X_5.values)

# Obtain cluster centers
cluster_centers = kmeans_model_rbf.get_centroids()
cluster_centers_df = pd.DataFrame(cluster_centers)
print("The cluster centers are:\n", cluster_centers_df)


# Show the summation of all dimensions of the cluster centroids
cluster_summations = [np.sum(each) for each in cluster_centers]
print("The cluster summations are:\n", cluster_summations)

The cluster centers are:
         0         1         2         3         4         5         6    \
0  0.389895  0.339402  0.395880  0.432120  0.533973  0.434016  0.442943   
1  0.295747  0.374546  0.283198  0.277281  0.281035  0.311879  0.273807   
2  0.400473  0.346565  0.351769  0.397071  0.380096  0.380866  0.399777   
3  0.377049  0.374207  0.342303  0.332540  0.382167  0.402853  0.414808   
4  0.412127  0.333143  0.382702  0.610461  0.466886  0.394356  0.582641   

        7         8         9    ...       490       491       492       493  \
0  0.367575  0.492517  0.506715  ...  0.323000  0.421278  0.426196  0.363953   
1  0.325118  0.282587  0.285214  ...  0.317200  0.270439  0.269456  0.306310   
2  0.406902  0.417931  0.375415  ...  0.380545  0.384639  0.387250  0.351074   
3  0.306453  0.422305  0.418517  ...  0.322910  0.316099  0.315702  0.360258   
4  0.334939  0.640038  0.450084  ...  0.391431  0.499408  0.609257  0.397160   

        494       495       496       497 